In [116]:
import csv
import nltk
from pymorphy2 import MorphAnalyzer

У нас есть 2 составленных нами словаря, с которыми производится сравнение: один (закомменченный) очень качественный, но дает мало результатов, второй (незакомменченный) дает много результатов, но часть из них мусор

In [117]:

d = {}

#with open ('results.csv', encoding = 'utf-8') as csvfile:
with open ('full_results.csv', encoding = 'utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter='\t')
        for row in readCSV:
            d[row[1]] = tuple((row[0], row[2]))


In [118]:
to_file = []

In [119]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words('russian')
russian_stopwords = [elem for elem in russian_stopwords if elem not in ['не', 'да', 'но', 'хорошо', 'лучше', 'почти', 'никогда', 'можно', 'наконец', 'больше', 'другой', 'всего', 'много', 'впрочем', 'нельзя', 'более', 'всегда']]

In [120]:
m = MorphAnalyzer()
tokenizer = nltk.RegexpTokenizer(r'\w+')

def normalize(text):
    tokens = tokenizer.tokenize(text.lower())
    lemmas = []
    pos = []
    for token in tokens:
        m.parse(token)[0]
        tag = m.parse(token)[0]
        lemmas.append(tag.normal_form)

    return lemmas

На вход поставляеится адресс конкретного текста  без парсинга и с парсингом

In [121]:
def analyze(norm_text, parsed_txt):
    to_file = []
    trees = []
    with open(parsed_txt, 'r', encoding = 'utf-8') as f:
        parsed_sents = f.read().split('\n\n')

        for sent in parsed_sents:
            tree = [line for line in sent.split('\n') if line and line[0] != '#']
            trees.append('\n'.join(tree))
            
        for index, tree in enumerate(trees[:-1]):
            if tree:
                
                tree = tree.split('\n')
                words = {}
                sent = [t.split('\t')[1] for t in tree]
                for t in tree:
                    words[t.split('\t')[2]] = t.split('\t')[0]
                sent = ' '.join(sent)
                sent = normalize(sent)
                sent = [word for word in sent if word not in russian_stopwords]

                for i in range(len(sent)):
                    for j in range(i+1, len(sent)):
                        
                        if ' '.join(sent[i:j]) in d.keys():
                            #print('yes')
                            try: # проблема возникает с токенами посреди которых есть точка, но нам такие токены все равно не нужны, ибо они все мусор, так что
                                to_file.append('\t'.join([str(index+1), ','.join([words[sent[i]], words[sent[j-1]]]), d[' '.join(sent[i:j])][0],  d[' '.join(sent[i:j])][1]]))
                            except KeyError:
                                continue
                                
    with open(norm_text[:-3] + 'tsv','w', encoding = 'utf-8') as f:
        f.write('\n'.join(to_file))

In [122]:
analyze('1032.txt', '1032.txt.parsed')